In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
pip list | grep -E 'transformers|tensorflow'

tensorflow                         2.17.0
tensorflow-datasets                4.9.6
tensorflow-hub                     0.16.1
tensorflow-io-gcs-filesystem       0.37.1
tensorflow-metadata                1.16.1
tensorflow-probability             0.24.0
transformers                       4.44.2


In [ ]:
pip install --upgrade transformers tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.3 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: transformers

In [ ]:
pip install transformers

In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from tqdm import tqdm, trange
from ast import literal_eval

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf


In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/FYP 2024/DATASET/women_final.csv') #-comment-classification-challenge
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/FYP 2024/DATASET/women_final.csv'

In [ ]:
df["text"][1]

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
print('Unique comments: ', df.text.nunique() == df.shape[0])
print('Null values: ', df.isnull().values.any())
df[df.isna().any(axis=1)]

NameError: name 'df' is not defined

In [ ]:
print('average sentence length: ', df.text.str.split().str.len().mean())
print('stdev sentence length: ', df.text.str.split().str.len().std())

In [ ]:
cols = df.columns
label_cols = list(cols[2:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)

In [ ]:
print('Count of 1 per label: \n', df[label_cols].sum(), '\n') # Label counts, may need to downsample or upsample
print('Count of 0 per label: \n', df[label_cols].eq(0).sum())

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df['one_hot_labels'] = list(df[label_cols].values)
df.head()

In [ ]:
labels = list(df.one_hot_labels.values)
comments = list(df.text.values)

In [ ]:
import os
current_dir = os.getcwd()
print(os.listdir(current_dir))

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
max_length = 100
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer
encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

NameError: name 'comments' is not defined

In [ ]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

In [ ]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df.one_hot_labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

In [ ]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

In [ ]:

train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks,
                                                            random_state=2020, test_size=0.30, stratify = labels)

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)
train_token_types.extend(one_freq_token_types)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

In [ ]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

In [ ]:
# Load model, the pretrained model will include a single linear classification layer on top for classification.
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.cuda()

In [ ]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
from transformers import AdamW


In [ ]:
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)
# optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

# Store our loss and accuracy for plotting
train_loss_set = []
train_f1_set = []
train_acc_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 5

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0  # running loss
    nb_tr_examples, nb_tr_steps = 0, 0
    train_preds, train_labels = [], []
    threshold=0.50

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_token_types = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()

        # Forward pass for multilabel classification
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]
        loss_func = BCEWithLogitsLoss()
        loss = loss_func(logits.view(-1, num_labels), b_labels.type_as(logits).view(-1, num_labels))  # convert labels to float for calculation
        train_loss_set.append(loss.item())

        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

        # Extend the lists with current batch predictions and labels
        pred_bools = [pl > threshold for pl in logits.detach().cpu().numpy()]
        train_preds.extend(pred_bools)
        train_labels.extend(b_labels.cpu().numpy())

    # Calculate train accuracy and F1 score
    train_acc = accuracy_score(train_labels, train_preds) * 100
    train_f1 = f1_score(train_labels, train_preds, average='macro') * 100

    train_f1_set.append(train_f1)
    train_acc_set.append(train_acc)

    print("Train loss: {}".format(tr_loss / nb_tr_steps))
    print("Train Accuracy: {:.2f}%".format(train_acc))

###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
model.eval()

  # Variables to gather full output
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
threshold = 0.50
pred_bools = [pl>threshold for pl in pred_labels]
true_bools = [tl==1 for tl in true_labels]

val_f1macro_accuracy = f1_score(true_bools,pred_bools,average='macro')*100
val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

print('Validation Accuracy: ', val_flat_accuracy)

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/FYP 2024/DATASET/women_final.csv')
test_label_cols = list(test_df.columns[2:])
print('Null values: ', test_df.isnull().values.any()) #should not be any null sentences or labels
print('Same columns between train and test: ', label_cols == test_label_cols) #columns should be the same
test_df.head()

In [ ]:
test_df = test_df[~test_df[test_label_cols].eq(-1).any(axis=1)] #remove irrelevant rows/comments with -1 values
test_df['one_hot_labels'] = list(test_df[test_label_cols].values)
test_df.head()

In [ ]:
# Gathering input data
test_labels = list(test_df.one_hot_labels.values)
test_comments = list(test_df.text.values)

In [ ]:
#Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True)
test_input_ids = test_encodings['input_ids']
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

In [ ]:
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_data_loader')

In [ ]:
# Test

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_token_types = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)

    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

In [ ]:

pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding


# Print and save classification report
print('Test F1 macro Accuracy: ', f1_score(true_bools, pred_bools,average='macro'))
print('Test Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
clf_report = classification_report(true_bools,pred_bools,target_names=test_label_cols)
pickle.dump(clf_report, open('classification_report.txt','wb')) #save report
print(clf_report)

In [ ]:
idx2label = dict(zip(range(12),label_cols))
print(idx2label)



In [ ]:
# Getting indices of where boolean one hot vector true_bools is True so we can use idx2label to gather label names
true_label_idxs, pred_label_idxs=[],[]
for vals in true_bools:
  true_label_idxs.append(np.where(vals)[0].flatten().tolist())
for vals in pred_bools:
  pred_label_idxs.append(np.where(vals)[0].flatten().tolist())

In [ ]:
# Gathering vectors of label names using idx2label
true_label_texts, pred_label_texts = [], []
for vals in true_label_idxs:
  if vals:
    true_label_texts.append([idx2label[val] for val in vals])
  else:
    true_label_texts.append(vals)

for vals in pred_label_idxs:
  if vals:
    pred_label_texts.append([idx2label[val] for val in vals])
  else:
    pred_label_texts.append(vals)

In [ ]:
import pickle

In [ ]:
vector_data = {'true_labels': true_label_texts, 'pred_labels': pred_label_texts}

with open('vector.pkl', 'wb') as f:
    pickle.dump(vector_data, f)


In [ ]:
# Decoding input ids to comment text
texts = [tokenizer.decode(text,skip_special_tokens=True,clean_up_tokenization_spaces=False) for text in tokenized_texts]

In [ ]:
# Converting lists to df
comparisons_df = pd.DataFrame({'text': texts, 'true_labels': true_label_texts, 'pred_labels':pred_label_texts})
comparisons_df.to_csv('comparisons.csv')
comparisons_df.head(50)

In [ ]:
# Calculate Accuracy - maximize F1 accuracy by tuning threshold values. First with 'macro_thresholds' on the order of e^-1 then with 'micro_thresholds' on the order of e^-2

macro_thresholds = np.array(range(1,10))/10

f1_results, flat_acc_results = [], []
for th in macro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_macro_th = macro_thresholds[np.argmax(f1_results)] #best macro threshold value

micro_thresholds = (np.array(range(10))/100)+best_macro_th #calculating micro threshold values

f1_results, flat_acc_results = [], []
for th in micro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='macro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_f1_idx = np.argmax(f1_results) #best threshold value

# Printing and saving classification report
print('Best Threshold: ', micro_thresholds[best_f1_idx])
print('Test F1 Accuracy: ', f1_results[best_f1_idx])
print('Test Flat Accuracy: ', flat_acc_results[best_f1_idx], '\n')

best_pred_bools = [pl>micro_thresholds[best_f1_idx] for pl in pred_labels]
clf_report_optimized = classification_report(true_bools,best_pred_bools, target_names=label_cols)
pickle.dump(clf_report_optimized, open('classification_report_optimized.txt','wb'))
print(clf_report_optimized)

In [ ]:
pip install optuna

In [ ]:
import numpy as np
import optuna
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Define objective function for optimization
def objective(trial):
    # Define macro_thresholds on the order of e^-1
    macro_threshold = trial.suggest_uniform('macro_threshold', 0.1, 0.9)

    # Calculate F1 score and accuracy for each macro_threshold
    f1_results, flat_acc_results = [], []
    for th in [macro_threshold]:
        pred_bools = [pl > th for pl in pred_labels]
        test_f1_accuracy = f1_score(true_bools, pred_bools, average='micro')
        test_flat_accuracy = accuracy_score(true_bools, pred_bools)
        f1_results.append(test_f1_accuracy)
        flat_acc_results.append(test_flat_accuracy)

    # Find the best macro threshold
    best_macro_th = macro_thresholds[np.argmax(f1_results)]

    # Calculate micro_thresholds on the order of e^-2
    micro_thresholds = np.linspace(best_macro_th, best_macro_th + 0.09, num=10)

    # Calculate F1 score and accuracy for each micro_threshold
    f1_results, flat_acc_results = [], []
    for th in micro_thresholds:
        pred_bools = [pl > th for pl in pred_labels]
        test_f1_accuracy = f1_score(true_bools, pred_bools, average='macro')
        test_flat_accuracy = accuracy_score(true_bools, pred_bools)
        f1_results.append(test_f1_accuracy)
        flat_acc_results.append(test_flat_accuracy)

    # Get the best F1 score index
    best_f1_idx = np.argmax(f1_results)

    # Return the negative of F1 score as Optuna aims to minimize the objective
    return -f1_results[best_f1_idx]

# Optimize using Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Get the best parameters
best_macro_threshold = study.best_params['macro_threshold']

# Use the best macro threshold to calculate micro thresholds
best_micro_thresholds = np.linspace(best_macro_threshold, best_macro_threshold + 0.09, num=10)

# Evaluate using the best micro thresholds
f1_results, flat_acc_results = [], []
for th in best_micro_thresholds:
    pred_bools = [pl > th for pl in pred_labels]
    test_f1_accuracy = f1_score(true_bools, pred_bools, average='macro')
    test_flat_accuracy = accuracy_score(true_bools, pred_bools)
    f1_results.append(test_f1_accuracy)
    flat_acc_results.append(test_flat_accuracy)

# Get the index of the best F1 score
best_f1_idx = np.argmax(f1_results)

# Print and save classification report
print('Best Threshold: ', best_micro_thresholds[best_f1_idx])
print('Test F1 Accuracy: ', f1_results[best_f1_idx])
print('Test Flat Accuracy: ', flat_acc_results[best_f1_idx], '\n')

best_pred_bools = [pl > best_micro_thresholds[best_f1_idx] for pl in pred_labels]
clf_report_optimized = classification_report(true_bools, best_pred_bools, target_names=label_cols)
with open('classification_report_optimized.txt', 'w') as f:
    f.write(clf_report_optimized)
print(clf_report_optimized)


In [ ]:
import pickle

# Save the Optuna study object
with open('study.pkl', 'wb') as f:
    pickle.dump(study, f)

# Save the best parameters
best_params = {
    'best_macro_threshold': best_macro_threshold,
    'best_micro_thresholds': best_micro_thresholds
}
with open('best_params.pkl', 'wb') as f:
    pickle.dump(best_params, f)


In [ ]:
import pickle

# Save the model, tokenizer, and optimizer parameters
torch.save(model.state_dict(), 'model.pth')  # Save the model
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)  # Save the tokenizer
with open('optimizer_params.pkl', 'wb') as f:
    pickle.dump(optimizer_grouped_parameters, f)  # Save the optimizer parameters


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pickle

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
model.load_state_dict(torch.load('model.pth'))
model.eval()

# Load the tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Tokenize the input text
text = "Heard learned counsel for the applicant and learned AGA for the State. By means of this application, the applicant, who is involved in Case Crime No. __ of __, under sections __ IPC, police station Kaptanganj, district Kushinagar, is seeking enlargement on bail during the trial. Submission made by learned counsel for the applicant is that FIR was registered under sections __ IPC against three accused persons: Smt. Kaushalya Devi (mother-in-law), Ajay Yadav (husband), and Rahul Yadav (dewar). Further, before issuing the release order, the sureties should be verified: (i) the applicant shall file an undertaking to the effect that he shall not seek any adjournment on the date fixed for evidence when the witnesses are present in court. In case of default of this condition, it shall be open for the trial court to treat it as abuse of liberty of bail and pass orders in accordance with law; (ii) the applicant shall remain present before the trial court on each date fixed, either personally."
inputs = tokenizer(text, padding=True, truncation=True, max_length=100, return_tensors="pt")

# Get the model predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to probabilities
probs = torch.nn.functional.softmax(logits, dim=1)[0]

# Print the prediction scores
print("Prediction scores:", probs)


In [ ]:
# Find the index of the maximum score
predicted_label_idx = torch.argmax(probs).item()

# Map the index to the label using idx2label
predicted_label = idx2label[predicted_label_idx]

# Print the predicted label and its corresponding penal code
print("Predicted label:", predicted_label)
